In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time

In [3]:
t0 = time()
df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1000.0 million rows : 93.22586369514465 seconds


In [5]:
%%time
imp = df['impressions'].to_numpy(np.int32)//scale

CPU times: user 739 ms, sys: 2.37 s, total: 3.1 s
Wall time: 3.1 s


In [6]:
%%time
ds = EntityList.from_series(df['user_id'])

CPU times: user 46.6 s, sys: 885 ms, total: 47.5 s
Wall time: 47.5 s


In [7]:
%%time
ds_unique = df['user_id'].unique()

CPU times: user 44 s, sys: 164 ms, total: 44.2 s
Wall time: 44.2 s


In [9]:
len(ds_unique)/1e6

25.66891

In [10]:
%%time
a = np.unique(df['user_id']._values)

CPU times: user 1min 31s, sys: 1.58 s, total: 1min 32s
Wall time: 1min 32s


In [11]:
scale = 100  # This is put here to stop int32 overflow errors


t0 = time()
impressions = df['impressions'].to_numpy(np.int32) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 47.73137378692627 seconds


In [12]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

what type of child do we have now? [51357 51357 51357 ... 80605 80605 80605] <class 'numpy.ndarray'>
what type of child do we have now? [51357 51357 51357 ... 80605 80605 80605] <class 'numpy.ndarray'>
Time make Private Syft Tensor: 1.1010844707489014 seconds


In [13]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into cicada... done!


In [14]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... checking asset types...                                                                                                                                    

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft
Uploading Tweets data- 1647967659.5960708: 100%|██| 7/7 [00:08<00:00,  1.23s/it]


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 39.8 s, sys: 10.1 s, total: 49.9 s
Wall time: 1min 5s


In [15]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1647935659.4255164,Tweets- 1B rows,"[""Tweets data- 1647935659.4255164""] ->",3ccd2351-5a0c-4ea2-8ad4-2a532a2df419
[1],Tweets data- 1647937494.7269194,Tweets- 1B rows,"[""Tweets data- 1647937494.7269194""] ->",679498a5-0555-4d9f-9ea0-342daed6ce6e
[2],Tweets data- 1647937579.130175,Tweets- 1B rows,"[""Tweets data- 1647937579.130175""] ->",b1582714-f88e-405f-a0e5-e577b7b0364c
[3],Tweets data- 1647965464.2495682,Tweets- 1M rows,"[""Tweets data- 1647965464.2495682""] ->",34e569f7-1fc1-4922-b30e-5b471c4808b8
[4],Tweets data- 1647966224.1908379,Tweets- 1B rows,"[""Tweets data- 1647966224.1908379""] ->",0f6da123-b8a5-4c3f-86f1-4d8a9ce5a3f9
[5],Tweets data- 1647967659.5960708,Tweets- 1B rows,"[""Tweets data- 1647967659.5960708""] ->",246a3acd-488c-4ecc-aa28-ca3472e39a73


In [16]:
data = domain_node.datasets[-1][name]

In [17]:
data

<TensorPointer -> cicada:5d0058f20b034c98a0369c35e42e36c2>

In [ ]:
%%time
sum_result = data.sum()
sum_result.block

In [16]:
sum_result

<TensorPointer -> cicada:f81589d366cc4c438db9c5840986f65a>

In [ ]:
sum

In [17]:
gtype(deser_data)

syft.core.tensor.autodp.gamma_tensor.GammaTensor

array(-942906630, dtype=int32)